# Capstone Project - Milestone 2
---

For the second milestone in the capstone project, we will be completing the following objectives:

- Missing value treatment
- Outlier treatment and transformation of dynamic variables
- Feature Engineering
- Building and refining our Linear Regression Model

We will keep this notebook distinct from the preliminary EDA that we completed in the first milestone session notebook, although we will perform some further EDA after preprocessing the data and examining some of the engineered features we will be creating and testing before we build our regression model. We begin by loading in and processing our data in the first section of this notebook.

## Section 1 - Data Preprocessing
---

In this section, we will load in and process the dataset. This will include treatment of missing values and outliers and discarding unneeded features. 

In [3]:
#import necessary libraries
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
import warnings

sns.set_style('darkgrid')
pd.set_option("display.max_rows", 500)
warnings.filterwarnings('ignore')
%matplotlib inline
plt.rcParams['figure.figsize'] = [16, 12]

In [5]:
#set path to data and import dataset into DF
fpath = 'C:/Users/evnca/Documents/DSBA/capstone/capstone_project_data/'
fname = 'house_price_data'
fext  = '.csv'

#make a copy of the dataset to leave the original raw data intact if needed
raw_data = pd.read_csv(os.path.join(fpath+fname+fext))
data     = raw_data.copy(deep=True)

In [8]:
#Review features and rename as needed for clarity
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   cid               21613 non-null  int64  
 1   dayhours          21613 non-null  object 
 2   price             21613 non-null  int64  
 3   room_bed          21505 non-null  float64
 4   room_bath         21505 non-null  float64
 5   living_measure    21596 non-null  float64
 6   lot_measure       21571 non-null  float64
 7   ceil              21571 non-null  object 
 8   coast             21612 non-null  object 
 9   sight             21556 non-null  float64
 10  condition         21556 non-null  object 
 11  quality           21612 non-null  float64
 12  ceil_measure      21612 non-null  float64
 13  basement          21612 non-null  float64
 14  yr_built          21612 non-null  object 
 15  yr_renovated      21613 non-null  int64  
 16  zipcode           21613 non-null  int64 

In [9]:
data.rename(columns={"dayhours"  : "sale_date",
                     "room_bed"  : "num_beds",
                     "room_bath" : "num_baths",
                     "ceil"      : "floors",
                     "sight"     : "viewed",
                    })

,cid,sale_date,price,num_beds,num_baths,living_measure,lot_measure,floors,coast,viewed,...,basement,yr_built,yr_renovated,zipcode,lat,long,living_measure15,lot_measure15,furnished,total_area
0,3876100940,20150427T000000,600000,4.0,1.75,3050.0,9440.0,1,0,0.0,...,1250.0,1966,0,98034,47.7228,-122.183,2020.0,8660.0,0.0,12490
1,3145600250,20150317T000000,190000,2.0,1.00,670.0,3101.0,1,0,0.0,...,0.0,1948,0,98118,47.5546,-122.274,1660.0,4100.0,0.0,3771
2,7129303070,20140820T000000,735000,4.0,2.75,3040.0,2415.0,2,1,4.0,...,0.0,1966,0,98118,47.5188,-122.256,2620.0,2433.0,0.0,5455
3,7338220280,20141010T000000,257000,3.0,2.50,1740.0,3721.0,2,0,0.0,...,0.0,2009,0,98002,47.3363,-122.213,2030.0,3794.0,0.0,5461
4,7950300670,20150218T000000,450000,2.0,1.00,1120.0,4590.0,1,0,0.0,...,0.0,1924,0,98118,47.5663,-122.285,1120.0,5100.0,0.0,5710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,203600600,20150310T000000,685530,4.0,2.50,3130.0,60467.0,2,0,0.0,...,0.0,1996,0,98014,47.6618,-121.962,2780.0,44224.0,1.0,63597
21609,625049281,20140521T000000,535000,2.0,1.00,1030.0,4841.0,1,0,0.0,...,110.0,1939,0,98103,47.6860,-122.341,1530.0,4944.0,0.0,5871
21610,424069018,20140905T000000,998000,3.0,3.75,3710.0,34412.0,2,0,0.0,...,800.0,1978,0,98075,47.5888,-122.04,2390.0,34412.0,1.0,38122
21611,7258200055,20150206T000000,262000,4.0,2.50,1560.0,7800.0,2,0,0.0,...,0.0,1997,0,98168,47.5140,-122.316,1160.0,7800.0,0.0,9360


In [10]:
#check for missing values and percentage of missing values
data.isnull().sum()

cid                   0
dayhours              0
price                 0
room_bed            108
room_bath           108
living_measure       17
lot_measure          42
ceil                 42
coast                 1
sight                57
condition            57
quality               1
ceil_measure          1
basement              1
yr_built              1
yr_renovated          0
zipcode               0
lat                   0
long                  0
living_measure15    166
lot_measure15        29
furnished            29
total_area           29
dtype: int64

In [14]:
#express as a percentage
data.isnull().sum() / data.shape[0] * 100

cid                 0.000000
dayhours            0.000000
price               0.000000
room_bed            0.499699
room_bath           0.499699
living_measure      0.078656
lot_measure         0.194327
ceil                0.194327
coast               0.004627
sight               0.263730
condition           0.263730
quality             0.004627
ceil_measure        0.004627
basement            0.004627
yr_built            0.004627
yr_renovated        0.000000
zipcode             0.000000
lat                 0.000000
long                0.000000
living_measure15    0.768056
lot_measure15       0.134179
furnished           0.134179
total_area          0.134179
dtype: float64